# Setting up and running PICASO 

Follow installation guide to set up reference files and opacities. 
[INSERT LINK]

In [1]:
import numpy as np
import pandas as pd
#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi

/Users/natashabatalha/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/natashabatalha/anaconda3/lib/python3.6/site-packages/pysynphot/locations.py:329: UserWarning: Extinction files not found in /Users/natashabatalha/pysynphot_data/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/Users/natashabatalha/anaconda3/lib/python3.6/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/natashabatalha/pysynphot_data/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/natashabatalha/anaconda3/lib/python3.6/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no therma

In [2]:
#if you are familiar with PandExo, this setup will be a familiar process 
all_inputs = jdi.load_inputs()

## Set Planet & Star Properties 

In [3]:
#gravity 
all_inputs['planet']['gravity'] = 25
all_inputs['planet']['gravity_unit'] = 'm/(s**2)' #any astropy units are okay here 
#can also enter in radius and mass 
#all_inputs['planet']['mass']=1
#all_inputs['planet']['mass_unit'] = 'M_earth' (radius would be the same but with 'radius' and 'radius_unit')

#add star properties 
all_inputs['star']['temp'] = 6000 #kelvin
all_inputs['star']['metal'] = 0.0122 #log metal
all_inputs['star']['logg'] = 4.437 #log cgs

## Define Atmospheric Composition

### Option 1) Specify file path 

In [4]:
all_inputs['atmosphere']['profile']['filepath'] = jdi.jupiter_pt() #this is a pointer to a preloaded PT profile 
                                                                   #to help guide us

### File format Specifications
1) Must have header which includes.. 
- At least **pressure**(bars), **temperature**(Kelvin), and `case sensitive` molecule names (e.g. TiO, Na, H2O, etc) for mixing ratios (in no particular order)

2) Must be white space delimeted 

#### PICASO will auto-compute mixing ratios, determine what CIA is neceesary and compute mean molecular weight based on these headers. Take at the preloaded example below

In [ ]:
comp_file = pd.read_csv(jdi.jupiter_pt(), delim_whitespace=True)
#see example below
comp_file.head()

### Or... Option 2) Give arbitrary pressure, temperature and mixing ratios for simple atmospheres


In [ ]:
all_inputs['atmosphere']['profile']['profile'] = pd.DataFrame({'pressure':np.logspace(-6,2,60),
                                                 'temperature':np.logspace(-6,2,60)*0+200,
                                                 "H2":np.logspace(-6,2,60)*0+0.837,
                                                 "He":np.logspace(-6,2,60)*0+0.163,
                                                 "CH4":np.logspace(-6,2,60)*0+0.000466})

## Run 

In [5]:
phase_angle = 0 #radians
wno, alb = jdi.picaso(all_inputs, phase_angle)

In [6]:
from bokeh.plotting import show
show(jpi.spectrum(wno, alb))

## Can Also Define subset of molecules 

Sometimes it's helpful to do a run and exclude one or several molecules from a bigger list. Take a look below

In [ ]:
all_inputs['atmosphere']['profile']['filepath'] = jdi.jupiter_pt() 

#let's say I want to remove everything but methane and H2/He to see if there were anything other than these that were 
#influencing the spectrum.

all_inputs['atmosphere']['molecules']['whichones'] = ['He', 'H2', 'CH4']

wno, alb = jdi.picaso(all_inputs, phase_angle)
show(jpi.spectrum(wno, alb))